# Issue with getPeopleByTrack
---
The REST API responds as expected in most cases.

In [1]:
import requests, json
session = requests.Session()
temporaryKey = 'Zjc3NTliODgyZjMwMDFiYjlkMmZkYTlmYjRkZTlhZjRraW1pMTYwMjI4NzcwNQ'
session.headers.update({'Authorization': f'Bearer {temporaryKey}'})

Bershtein Law CEO approved the public release of this API key for troubleshooting, since it is tied to a temporary/test company.

The company is returned as expected:

In [2]:
companyResponse = session.get('https://app.trackwithease.com/api/api/getCompany')
print(companyResponse.status_code)
print(companyResponse.json()['result'][0]['name'])

200
Northeast Corridor Productions


In [3]:
seller, buyer = 'Stark', 'Wayne'
street, town, state, zipCode = '10880 Malibu Point', 'Malibu', 'California', '90265'
addTrackResponse = session.post('https://app.trackwithease.com/api/api/add_track',
                                {'name': f'{seller} to {buyer}; {street}, {town}, {state} {zipCode}',
                                 'folder': 0})
print(addTrackResponse.status_code)

200


The status code appears to always be returned as `200` since `response.json()['error']` is used instead. Created tracks seem to be named properly:

In [4]:
trackId = addTrackResponse.json()['result']['track_id']
getAllTracksResponse = session.get('https://app.trackwithease.com/api/api/getTracks')
for track in getAllTracksResponse.json()['result']:
    print(track['name'])


Stark to Wayne; 10880 Malibu Point, Malibu, California 90265


Information can be added to the track profile:

In [5]:
addressKey = 'property_address'
updateTrackProfileResponse = session.post('https://app.trackwithease.com/api/api/updateTrackProfile',
                                          {'track_id': trackId,
                                           f'{addressKey}[address]': street,
                                           f'{addressKey}[city]': town,
                                           f'{addressKey}[state]': state,
                                           f'{addressKey}[zip_code]': zipCode,
                                           f'{addressKey}[country]': 'United States'})
getTrackProfileResponse = session.post('https://app.trackwithease.com/api/api/getTrackProfile',
                                       {'track': trackId})
print(getTrackProfileResponse.json()['result'][addressKey])

{'address': '10880 Malibu Point', 'city': 'Malibu', 'state': 'California', 'zip_code': '90265', 'country': 'United States'}


All people from the company can be acquired:

In [6]:
getAllPeopleResponse = session.get('https://app.trackwithease.com/api/api/getPeople')
for personInfo in getAllPeopleResponse.json()['result']:
    print(personInfo['first_name'])

Andrew


Acquiring people by track still returns a `200` status:

In [7]:
getTrackPeopleResponse = session.post('https://app.trackwithease.com/api/api/getPeopleByTrack',
                                      {'track': trackId})
print(getTrackPeopleResponse.status_code)

200


Can't get `error` code back from JSON object if response object cannot convert to JSON:

In [8]:
try:
    print(getTrackPeopleResponse.json()['error'])
except json.JSONDecodeError as error:
    print(f'JSONDecodeError raised because...{error}')

JSONDecodeError raised because...Expecting value: line 2 column 1 (char 1)


Issue does not appear to be JSON specific:

In [9]:
print(f'{getTrackPeopleResponse.text[:1222]}...')


<div style="border:1px solid #990000;padding-left:20px;margin:0 0 10px 0;">

<h4>A PHP Error was encountered</h4>

<p>Severity: Notice</p>
<p>Message:  Undefined variable: people</p>
<p>Filename: api/Api.php</p>
<p>Line Number: 556</p>


	<p>Backtrace:</p>
	
		
	
		
	
		
			<p style="margin-left:10px">
			File: /home/twe15bstone/public_html/application/controllers/api/Api.php<br />
			Line: 556<br />
			Function: _error_handler			</p>

		
	
		
	
		
			<p style="margin-left:10px">
			File: /home/twe15bstone/public_html/index.php<br />
			Line: 293<br />
			Function: require_once			</p>

		
	

</div>
<div style="border:1px solid #990000;padding-left:20px;margin:0 0 10px 0;">

<h4>A PHP Error was encountered</h4>

<p>Severity: Notice</p>
<p>Message:  Trying to get property of non-object</p>
<p>Filename: api/Api.php</p>
<p>Line Number: 556</p>


	<p>Backtrace:</p>
	
		
	
		
	
		
			<p style="margin-left:10px">
			File: /home/twe15bstone/public_html/application/controllers/api/Api.php<br /